In [1]:
import numpy as np
from cmdstanpy import CmdStanModel
import math
import warnings
warnings.filterwarnings('ignore')

2. A multiple-choice test item has four options. Assume that a student taking this question either knows the answer or does a pure guess. A random sample of 100 students take the item. 60% get it correct. Give an estimate and 95% confidence interval for the percentage in the population who know the answer.

### Analytical

In [124]:
#0.6 = 0.25 + 0.75theta
estimate = (0.6 - 0.25) / 0.75
estimate

0.4666666666666666

In [125]:
#0.6 = 0.25 + 0.75theta
# 1.96 se's in each direction for 0.025/0.975
se = 1.96 * (math.sqrt((estimate * (1 - estimate)) / 100)) / 0.75
se

0.13037597294358977

In [123]:
print(estimate - se, estimate, estimate + se)

0.33629069372307685 0.4666666666666666 0.5970426396102564


### Stan

In [ ]:
'''
data {
    int N;
}
transformed data {
    real theta = 0.25;
}
generated quantities {
    vector[N] y_sim;
    for(i in 1:N) {
        y_sim[i] = binomial_rng(N - i, theta) + i;
    }
}
'''

In [108]:
# n test takers
N = 100
bino_model = CmdStanModel(stan_file='exercise2.stan')
bino_fit = bino_model.sample(output_dir='./output', data={'N': N}, fixed_param=True)

# 90 ci result:
#0.05%: 38%, mean: 47%, 95%: 53%,
for i, v in enumerate(bino_fit.summary()['Mean']):
    if v > (0.595 * N) and v < (0.605 * N):
        print(f'{i}% takers who know the answer results in total correct % (We need 60%):',round(v, 2))

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:compiled model file: /Users/spetulla/Workspace/Sandbox/ds-practice/Regression/exercise2
INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


46% takers who know the answer results in total correct % (We need 60%): 59.57
47% takers who know the answer results in total correct % (We need 60%): 60.34
